In [1]:
import numpy as np
import pandas as pd

In [2]:
all_matches = pd.read_csv(r'/home/chetan/Downloads/ipl_csv2/all_matches.csv')
all_matches.head()

/home/chetan/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Preparing the Statistics of Batsmen and Bowlers from Past Data

In [3]:
def get_batsmen_statistics(matches_df):

    batsmen_df = pd.DataFrame(columns = ['Batsman','Runs','Balls_Faced','Fours','Sixes','Strike_Rate'])
    strikers = matches_df.striker.unique()
    for striker in strikers:    
        batsman_ball2ball = matches_df[matches_df.striker==striker].runs_off_bat
        runs = batsman_ball2ball.sum()
        fours = int(batsman_ball2ball[batsman_ball2ball==4].count())
        sixes = int(batsman_ball2ball[batsman_ball2ball==6].count())
        balls = int(batsman_ball2ball.shape[0])
        strike_rate = round(runs/balls*100,2)
        batsmen_df=batsmen_df.append({'Batsman':striker,
                                      'Runs':runs,
                                      'Balls_Faced':balls,
                                      'Fours':fours,
                                      'Sixes':sixes,
                                      'Strike_Rate':strike_rate},ignore_index=True)
    
    return batsmen_df

def get_bowlers_statistics(matches_df):
    
    matches_df = matches_df.iloc[:]
    matches_df.fillna(0,inplace=True)
    bowlers_df = pd.DataFrame(columns = ['Bowler','Runs_Conceded','Balls','Economy_Rate'])
    bowlers = matches_df.bowler.unique()
    matches_df['total_runs'] = matches_df['runs_off_bat']+matches_df['wides']+matches_df['noballs']
    
    for bowler in bowlers:
        bowler_ball2ball = matches_df[matches_df.bowler==bowler]
        runs_conceded = bowler_ball2ball.total_runs.sum()

        #calculates the total number of wide balls and noballs
        wide_balls = bowler_ball2ball[bowler_ball2ball.wides !=0].shape[0]
        noballs = bowler_ball2ball[bowler_ball2ball.noballs !=0].shape[0]
        extra_balls = wide_balls+noballs
        
        #calculates the total number of overs
        balls = bowler_ball2ball.shape[0]
        overs = (balls-extra_balls)/6
        
        #economy rate
        economy_rate = round(runs_conceded/overs,2)
        
        bowlers_df = bowlers_df.append({'Bowler':bowler,'Economy_Rate':economy_rate,
                                        'Runs_Conceded':runs_conceded,'Balls':balls-extra_balls},ignore_index = True)
        
    return bowlers_df

In [4]:
all_matches.fillna(0,inplace=True)

#PP stands for Powerplay
all_matches_PP = all_matches[all_matches.ball<5.99]

In [5]:
batsmen_database_PP = get_batsmen_statistics(all_matches_PP)
batsmen_database_PP

,Batsman,Runs,Balls_Faced,Fours,Sixes,Strike_Rate
0,SC Ganguly,529,540,76,9,97.96
1,BB McCullum,1557,1278,184,70,121.83
2,RT Ponting,45,75,2,1,60.00
3,R Dravid,905,855,148,7,105.85
4,W Jaffer,77,95,9,1,81.05
...,...,...,...,...,...,...
284,RD Gaikwad,71,77,6,3,92.21
285,PK Garg,17,13,0,2,130.77
286,JO Holder,4,4,1,0,100.00
287,Washington Sundar,10,17,0,0,58.82


In [6]:
bowlers_database_PP = get_bowlers_statistics(all_matches_PP)
bowlers_database_PP

,Bowler,Runs_Conceded,Balls,Economy_Rate
0,P Kumar,1754.0,1572,6.69
1,Z Khan,1452.0,1292,6.74
2,AA Noffke,22.0,6,22.00
3,AB Dinda,991.0,810,7.34
4,I Sharma,1444.0,1215,7.13
...,...,...,...,...
341,CJ Green,16.0,12,8.00
342,DR Sams,38.0,30,7.60
343,Monu Kumar,7.0,6,7.00
344,KA Jamieson,7.0,12,3.50


In [ ]:
#Running the followling lines of code with save the statistics as csv files

#batsmen_database_PP.to_csv('batsmen_database_PP.csv')
#bowlers_database_PP.to_csv('bowlers_database_PP.csv')

# Data Preprocessing

In [7]:
data_df = all_matches.copy()

#filling nan with 0
data_df.fillna(0,inplace=True)

#considering only the powerplay
data_df = data_df[data_df.ball<5.99]

#eliminating super overs
data_df = data_df[data_df.innings<=2] 

#dropping useless columns
data_df.drop(['start_date','non_striker','wides','noballs','byes','legbyes',
              'penalty','other_wicket_type','other_player_dismissed'],axis=1,inplace=True)

#adding total runs column
data_df['total_runs']= data_df['runs_off_bat']+data_df['extras']



In [8]:
data_df

,match_id,season,venue,innings,ball,batting_team,bowling_team,striker,bowler,runs_off_bat,extras,wicket_type,player_dismissed,total_runs
0,335982,2007/08,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,0,1,0,0,1
1,335982,2007/08,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,0,0,0,0
2,335982,2007/08,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,1,0,0,1
3,335982,2007/08,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,0,0,0,0
4,335982,2007/08,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194260,1254060,2021,"MA Chidambaram Stadium, Chepauk, Chennai",2,5.2,Sunrisers Hyderabad,Kolkata Knight Riders,JM Bairstow,PJ Cummins,0,0,0,0,0
194261,1254060,2021,"MA Chidambaram Stadium, Chepauk, Chennai",2,5.3,Sunrisers Hyderabad,Kolkata Knight Riders,JM Bairstow,PJ Cummins,1,0,0,0,1
194262,1254060,2021,"MA Chidambaram Stadium, Chepauk, Chennai",2,5.4,Sunrisers Hyderabad,Kolkata Knight Riders,MK Pandey,PJ Cummins,0,0,0,0,0
194263,1254060,2021,"MA Chidambaram Stadium, Chepauk, Chennai",2,5.5,Sunrisers Hyderabad,Kolkata Knight Riders,MK Pandey,PJ Cummins,0,0,0,0,0


In [22]:
data_df_processed = pd.DataFrame(columns = ['match_id','season','venue','innings','batting_team','bowling_team','batsmen','bowlers',
                                   'avg_strike_rate','avg_economy_rate','wickets','runs'])

In [23]:
data_df_processed

,match_id,season,venue,innings,batting_team,bowling_team,batsmen,bowlers,avg_strike_rate,avg_economy_rate,wickets,runs


In [24]:
#returns the avg economy rate of the list of bowlers passed as an argument

def get_avg_economy_rate(bowlers):
    
    if len(bowlers)==0:
        return 7.0
    
    economy_t = 0
    for bowler in bowlers:
        if bowler in bowlers_database_PP.Bowler.values:
            index_of_bowler = bowlers_database_PP[bowlers_database_PP.Bowler==bowler].index.values
            economy = float(bowlers_database_PP.Economy_Rate.iloc[index_of_bowler])
        else:
            economy = 7.0
           
        economy_t += economy
    return round(economy_t/len(bowlers),2)

#***************************************************
#function returns the average strike rate of the list of batsmen passed as an argument

def get_avg_strike_rate(batsmen):
    
    if len(batsmen)==0:
        return 100
    strike_rate_t=0  #stores the total strike rate
    for batsman in batsmen:
        if batsman in batsmen_database_PP.Batsman.values:
            index_of_batsman = batsmen_database_PP[batsmen_database_PP.Batsman==batsman].index.values
            strike_rate = float(batsmen_database_PP.Strike_Rate.iloc[index_of_batsman])
            
        else:    #if the batsman is new, then assuming his strikerate is 100
            strike_rate = 100
        
        strike_rate_t += strike_rate
            
    return round(strike_rate_t/len(batsmen),2)

In [25]:
match_ids = data_df.match_id.unique()

for match_id in match_ids:
    
    match_df = data_df[data_df.match_id==match_id]
    match_deets = match_df.iloc[0]
    season = match_deets.season
    venue = match_deets.venue
    batting_team = match_deets.batting_team
    bowling_team = match_deets.bowling_team
    
    for i in [1,2]:
        
        if i==2:
            (batting_team,bowling_team) = (bowling_team,batting_team)
        
        innings_df = match_df[match_df.innings==i]
        runs = innings_df.total_runs.sum()
        #calculating the number of wickets
        wicket_balls = innings_df[innings_df.player_dismissed != 0]
        wickets = wicket_balls.shape[0]

        batsmen = innings_df.striker.unique()
        bowlers = innings_df.bowler.unique()

        avg_strike_rate = get_avg_strike_rate(batsmen)
        avg_economy_rate = get_avg_economy_rate(bowlers)

        row_dict = {'match_id':match_id,'season':season,'venue':venue,'innings':i,'batting_team':batting_team,
                    'bowling_team':bowling_team,'batsmen':batsmen,'bowlers':bowlers,
                    'avg_strike_rate':avg_strike_rate,'avg_economy_rate':avg_economy_rate,
                    'wickets':wickets,'runs':runs}

        
        data_df_processed=data_df_processed.append(row_dict,ignore_index=True)

In [26]:
data_df_processed.head()

,match_id,season,venue,innings,batting_team,bowling_team,batsmen,bowlers,avg_strike_rate,avg_economy_rate,wickets,runs
0,335982,2007/08,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,"[SC Ganguly, BB McCullum, RT Ponting]","[P Kumar, Z Khan, AA Noffke]",93.26,11.81,1,61
1,335982,2007/08,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,"[R Dravid, W Jaffer, V Kohli, JH Kallis, CL Wh...","[AB Dinda, I Sharma, AB Agarkar]",96.92,7.99,4,26
2,335983,2007/08,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Kings XI Punjab,"[PA Patel, ML Hayden, MEK Hussey]","[B Lee, S Sreesanth, JR Hopes]",115.25,7.64,1,53
3,335983,2007/08,"Punjab Cricket Association Stadium, Mohali",2,Kings XI Punjab,Chennai Super Kings,"[K Goel, JR Hopes]","[JDP Oram, MS Gony]",108.70,9.14,1,63
4,335984,2007/08,Feroz Shah Kotla,1,Rajasthan Royals,Delhi Daredevils,"[T Kohli, YK Pathan, SR Watson, M Kaif]","[GD McGrath, B Geeves, MF Maharoof]",90.31,8.54,2,40


In [27]:
#Run the following line of code to save the processed data

data_df_processed.to_csv('data_df_processed.csv',index=False)

# Preparing the Data for the Model

In [32]:
data_df_processed = pd.read_csv('data_df_processed.csv')
data_df_processed

,match_id,season,venue,innings,batting_team,bowling_team,batsmen,bowlers,avg_strike_rate,avg_economy_rate,wickets,runs
0,335982,2007/08,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,['SC Ganguly' 'BB McCullum' 'RT Ponting'],['P Kumar' 'Z Khan' 'AA Noffke'],93.26,11.81,1,61
1,335982,2007/08,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,['R Dravid' 'W Jaffer' 'V Kohli' 'JH Kallis' '...,['AB Dinda' 'I Sharma' 'AB Agarkar'],96.92,7.99,4,26
2,335983,2007/08,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Kings XI Punjab,['PA Patel' 'ML Hayden' 'MEK Hussey'],['B Lee' 'S Sreesanth' 'JR Hopes'],115.25,7.64,1,53
3,335983,2007/08,"Punjab Cricket Association Stadium, Mohali",2,Kings XI Punjab,Chennai Super Kings,['K Goel' 'JR Hopes'],['JDP Oram' 'MS Gony'],108.70,9.14,1,63
4,335984,2007/08,Feroz Shah Kotla,1,Rajasthan Royals,Delhi Daredevils,['T Kohli' 'YK Pathan' 'SR Watson' 'M Kaif'],['GD McGrath' 'B Geeves' 'MF Maharoof'],90.31,8.54,2,40
...,...,...,...,...,...,...,...,...,...,...,...,...
1633,1254058,2021,"MA Chidambaram Stadium, Chepauk, Chennai",2,Royal Challengers Bangalore,Mumbai Indians,['Washington Sundar' 'V Kohli' 'RM Patidar' 'G...,['TA Boult' 'JJ Bumrah' 'M Jansen' 'KH Pandya'],95.10,8.49,2,46
1634,1254059,2021,"Wankhede Stadium, Mumbai",1,Chennai Super Kings,Delhi Capitals,['RD Gaikwad' 'F du Plessis' 'MM Ali' 'SK Raina'],['CR Woakes' 'Avesh Khan' 'R Ashwin' 'TK Curran'],109.95,8.91,2,33
1635,1254059,2021,"Wankhede Stadium, Mumbai",2,Delhi Capitals,Chennai Super Kings,['PP Shaw' 'S Dhawan'],['DL Chahar' 'SM Curran' 'SN Thakur'],126.89,8.10,0,65
1636,1254060,2021,"MA Chidambaram Stadium, Chepauk, Chennai",1,Kolkata Knight Riders,Sunrisers Hyderabad,['N Rana' 'Shubman Gill'],['B Kumar' 'Sandeep Sharma' 'T Natarajan' 'Moh...,115.76,6.42,0,50


In [31]:
#label encoding the venue and teams
#RUN THIS BLOCK OF CODE ONLY ONCE
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

data_df_final = data_df_processed.copy()
venue_encoder = LabelEncoder()
team_encoder = LabelEncoder()

data_df_final['venue'] = venue_encoder.fit_transform(data_df_final.venue)

team = team_encoder.fit(data_df_final.batting_team)
data_df_final['batting_team'] = team.transform(data_df_final.batting_team)
data_df_final['bowling_team'] = team.transform(data_df_final.bowling_team)

data_df_final.head()

,match_id,season,venue,innings,batting_team,bowling_team,batsmen,bowlers,avg_strike_rate,avg_economy_rate,wickets,runs
0,335982,2007/08,15,1,7,13,['SC Ganguly' 'BB McCullum' 'RT Ponting'],['P Kumar' 'Z Khan' 'AA Noffke'],93.26,11.81,1,61
1,335982,2007/08,15,2,13,7,['R Dravid' 'W Jaffer' 'V Kohli' 'JH Kallis' '...,['AB Dinda' 'I Sharma' 'AB Agarkar'],96.92,7.99,4,26
2,335983,2007/08,27,1,0,5,['PA Patel' 'ML Hayden' 'MEK Hussey'],['B Lee' 'S Sreesanth' 'JR Hopes'],115.25,7.64,1,53
3,335983,2007/08,27,2,5,0,['K Goel' 'JR Hopes'],['JDP Oram' 'MS Gony'],108.70,9.14,1,63
4,335984,2007/08,9,1,10,3,['T Kohli' 'YK Pathan' 'SR Watson' 'M Kaif'],['GD McGrath' 'B Geeves' 'MF Maharoof'],90.31,8.54,2,40


In [33]:
chosen_features = ['venue','innings','batting_team','bowling_team','avg_strike_rate',
                   'avg_economy_rate','wickets']

#converting the dataframe into a numpy array
X = data_df_final[chosen_features].values
y = data_df_final.runs.values

#making the data type compatible for model training
X=X.astype(np.float32)
y=y.astype(np.float32)

print(X.shape)
print(y.shape)

(1638, 7)
(1638,)


In [34]:
#scaling the data

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Model Training

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [75]:
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y)

In [100]:
#Accuracy metric functions

def custom_accuracy_metric(y_true,y_pred,threshhold = 3):
    true_mat = y_pred[np.abs(y_true-y_pred)<=threshhold]
    return len(true_mat)/len(y_pred)

def analyse_model(model,X_train,X_test,y_train,y_test,threshhold=3):
    
    y_pred_train = np.array(list(map(int,model.predict(X_train))))
    y_pred_test = np.array(list(map(int,model.predict(X_test))))
    
    print('R2 Score for training set:',r2_score(y_train,y_pred_train))
    print('R2 Score for test set:',r2_score(y_test,y_pred_test))
    
    #using custom accuracy metric
    print('Accuracy for training set: ',custom_accuracy_metric(y_train,y_pred_train,threshhold))
    print('Accuracy for test set: ',custom_accuracy_metric(y_test,y_pred_test,threshhold))

Linear Regression Model

In [111]:
#lr stands for linear regression
lr = LinearRegression()
lr.fit(X_train,y_train)

print('**** Linear Regression ****')
analyse_model(lr,X_train,X_test,y_train,y_test)

**** Linear Regression ****
R2 Score for training set: 0.3063219363809446
R2 Score for test set: 0.3038022829901964
Accuracy for training set:  0.28827361563517917
Accuracy for test set:  0.3048780487804878


In [110]:
tree = DecisionTreeRegressor()
tree.fit(X_train,y_train)

print('**** Decision Tree Regressor ****')
analyse_model(tree,X_train,X_test,y_train,y_test)

**** Decision Tree Regressor ****
R2 Score for training set: 1.0
R2 Score for test set: -0.43848688260363833
Accuracy for training set:  1.0
Accuracy for test set:  0.2146341463414634


In [109]:
br = BayesianRidge()
br.fit(X_train,y_train)

print('**** Bayesian Ridge ****')
analyse_model(br,X_train,X_test,y_train,y_test)

**** Bayesian Ridge ****
R2 Score for training set: 0.3072552893719618
R2 Score for test set: 0.30495986800946284
Accuracy for training set:  0.28583061889250816
Accuracy for test set:  0.3


# Neural Network Model

In [78]:
from tensorflow import keras
from tensorflow.keras.layers import Dense

In [85]:
DL_model = keras.Sequential()
DL_model.add(Dense(64,input_dim=7, activation= "relu"))
DL_model.add(Dense(32, activation= "relu"))
DL_model.add(Dense(1))

DL_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                512       
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 2,625
Trainable params: 2,625
Non-trainable params: 0
_________________________________________________________________


In [86]:
DL_model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

DL_model.fit(X_train,y_train,epochs=20)

Epoch 1/20
39/39 [==============================] - 1s 3ms/step - loss: 2186.8192 - mean_squared_error: 2186.8192
Epoch 2/20
39/39 [==============================] - 0s 3ms/step - loss: 2007.9948 - mean_squared_error: 2007.9948
Epoch 3/20
39/39 [==============================] - 0s 3ms/step - loss: 1380.5866 - mean_squared_error: 1380.5866
Epoch 4/20
39/39 [==============================] - 0s 3ms/step - loss: 465.7325 - mean_squared_error: 465.7325
Epoch 5/20
39/39 [==============================] - 0s 2ms/step - loss: 211.0327 - mean_squared_error: 211.0327
Epoch 6/20
39/39 [==============================] - 0s 2ms/step - loss: 190.9479 - mean_squared_error: 190.9479
Epoch 7/20
39/39 [==============================] - 0s 2ms/step - loss: 191.5258 - mean_squared_error: 191.5258
Epoch 8/20
39/39 [==============================] - 0s 2ms/step - loss: 180.2466 - mean_squared_error: 180.2466
Epoch 9/20
39/39 [==============================] - 0s 2ms/step - loss: 157.0033 - mean_squared_er

In [113]:
print('**** Neural Network with 1 hidden layer ****')
analyse_model(DL_model,X_train,X_test,y_train,y_test,threshhold=2)

**** Neural Network with 1 hidden layer ****
R2 Score for training set: 0.2869520505131482
R2 Score for test set: 0.29811218197999945
Accuracy for training set:  0.18403908794788273
Accuracy for test set:  0.2146341463414634


In [ ]:
import joblib


joblib.dump(venue_encoder,'venue_encoder.joblib')
joblib.dump(team_encoder,'team_encoder.joblib')
joblib.dump(scaler,'scaler.joblib')
model.save('DL_model.h5')
joblib.dump(lr,'linear_regression.joblib')


In [ ]:
import pandas as pd
testInput = pd.read_csv('16_inn2.csv')
testInput